In [2]:
#todo do this same operation, where you create an id from individual to household, on every dataset which contains both variables, then save those datasets to "processed/individual_id_questions"

In [3]:
import pandas as pd
import os
import random
import math
import uuid
from typing import Optional
from collections.abc import Callable
from cleaning_functions.clean_csv import *
import numpy as np

Household to individual linking table

In [4]:
# Combine all files with households and individuals in one table
# note that individual ids are not unique. They reset for each new househould starting at 1
# we will need to create our own invidividual ids
path_to_files = "../data/raw"
csvs = os.listdir(path_to_files)
dfs = []
for file_name in csvs:
    df = pd.read_csv(path_to_files + "/" + file_name)
    try:
        df = df[["hhid", "indiv", ]]
        df.set_index("indiv", inplace=True)
        dfs.append(df)
    except KeyError:
        continue

combined_df = pd.concat(dfs, axis=0)
combined_df.reset_index(inplace=True)
#combined_df.drop_duplicates(subset=["hhid", "sector", "ea"], keep="first", inplace=True)
combined_df

/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/634710397.py:8: DtypeWarning: Columns (10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/634710397.py:8: DtypeWarning: Columns (16,36,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/634710397.py:8: DtypeWarning: Columns (28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)


,indiv,hhid
0,1.0,10001
1,2.0,10001
2,3.0,10001
3,4.0,10001
4,8.0,10001
...,...,...
196250,1.0,379155
196251,2.0,379155
196252,3.0,379155
196253,4.0,379155


In [5]:
combined_df.drop_duplicates(inplace=True)
combined_df

,indiv,hhid
0,1.0,10001
1,2.0,10001
2,3.0,10001
3,4.0,10001
4,8.0,10001
...,...,...
189939,1601.0,209113
190134,1601.0,219049
190135,1602.0,219049
193474,1601.0,299015


In [6]:
identifying_columns = ["hhid", "indiv"]
index_name = "individual_id"
set_unique_index(df=combined_df, columns_to_hash = identifying_columns, index_length_limit=12, index_name=index_name)
combined_df

,indiv,hhid
individual_id,,
f1f0cf89a4ed,1.0,10001
30f515782f82,2.0,10001
068d9042a81d,3.0,10001
88a95dd65b02,4.0,10001
b68864634b74,8.0,10001
...,...,...
34454fb5c5a6,1601.0,209113
1e8f93ed5b11,1601.0,219049
0e26fe36ed1a,1602.0,219049


In [7]:
combined_df.rename(columns = {"hhid": "household_id"}, inplace=True)
combined_df = combined_df[["household_id"]]

In [8]:
combined_df.to_csv("../data/processed/linking_tables/individual_id_to_household_id")

household to phone number linking table

In [9]:
# get a list of datasets which contain individual and household, and create the relationship from the linking table for each of them.
# save each df in a list
path_to_files = "../data/raw"
csvs = os.listdir(path_to_files)
dfs = []
identifying_columns = ["hhid", "indiv"]
index_name = "individual_id"

for file_name in csvs:
    df = pd.read_csv(path_to_files + "/" + file_name)
    try:
        df2 = df[["hhid", "indiv", ]]
        set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12, index_name=index_name, set_index=True)
        dfs.append((df, file_name))
    except KeyError:
        continue

/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/1535863809.py:10: DtypeWarning: Columns (10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/1535863809.py:10: DtypeWarning: Columns (16,36,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)
/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/1535863809.py:10: DtypeWarning: Columns (28,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_files + "/" + file_name)


In [10]:
dfs[0][1]

'p2r7_sect_2.csv'

In [11]:
working_df = dfs[0][0]
working_df

,zone,state,lga,sector,ea,hhid,indiv,s2q2,s2q3,s2q4,...,s2q5,s2q6,s2q7a,s2q7,s2q7_os,s2q8,s2q8_os,s2q9,s2q9_os,selected_farmer
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,2,1.0,NaN,...,1.0,63.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,2,1.0,NaN,...,2.0,59.0,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,2,1.0,NaN,...,2.0,13.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,2,1.0,NaN,...,2.0,54.0,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,2,1.0,NaN,...,1.0,10.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,2,1.0,NaN,...,1.0,49.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
e102508752e8,1,37,3702,1,1102,379155,2,2,1.0,NaN,...,2.0,45.0,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,2,1.0,NaN,...,2.0,9.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
rename_dict= {'s2q2': 'is_new_household_member',
              's2q3': 'is_still_household_member',
              's2q5': 'sex',
              's2q6': 'age',
              's2q7': 'relationship_to_head',
              's2q7_os': 'other_relationship',
              's2q8': 'why_join_household',
              's2q8_os':'other_reason_for_joining',
              'selected_farmer': 'is_selected_farmer'
              }
working_df = working_df.rename(columns = rename_dict)
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,sex,age,s2q7a,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,s2q9,s2q9_os,is_selected_farmer
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,2,1.0,NaN,...,1.0,63.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,2,1.0,NaN,...,2.0,59.0,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,2,1.0,NaN,...,2.0,13.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,2,1.0,NaN,...,2.0,54.0,1,12.0,NaN,NaN,NaN,NaN,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,2,1.0,NaN,...,1.0,10.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,2,1.0,NaN,...,1.0,49.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
e102508752e8,1,37,3702,1,1102,379155,2,2,1.0,NaN,...,2.0,45.0,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,2,1.0,NaN,...,2.0,9.0,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
working_df = working_df[[item for item in rename_dict.values()]]
working_df

,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,2,1.0,1.0,63.0,1.0,NaN,NaN,NaN,NaN
58b6880790bd,2,1.0,2.0,59.0,2.0,NaN,NaN,NaN,NaN
d2a2b4eb1d80,2,1.0,2.0,13.0,3.0,NaN,NaN,NaN,NaN
9dadc3d1eff3,2,1.0,2.0,54.0,12.0,NaN,NaN,NaN,NaN
cd696cdeb3ac,2,1.0,1.0,10.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,2,1.0,1.0,49.0,1.0,NaN,NaN,NaN,NaN
e102508752e8,2,1.0,2.0,45.0,2.0,NaN,NaN,NaN,NaN
b93d164cbee8,2,1.0,2.0,9.0,3.0,NaN,NaN,NaN,NaN


In [14]:
def recode_binary(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "YES"
        elif value == 2:
            return "NO"
        else:
            return np.nan

In [15]:
binary_columns = ['is_new_household_member',
                  'is_still_household_member',
                  'is_selected_farmer']
clean_columns(working_df, selected_columns=binary_columns, cleaning_functions_list=[recode_binary])
working_df

/Users/kathrynpanger/Documents/swb_inspect_data/cleaning_functions/clean_csv.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: function(x))


,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,NO,YES,1.0,63.0,1.0,NaN,NaN,NaN,NaN
58b6880790bd,NO,YES,2.0,59.0,2.0,NaN,NaN,NaN,NaN
d2a2b4eb1d80,NO,YES,2.0,13.0,3.0,NaN,NaN,NaN,NaN
9dadc3d1eff3,NO,YES,2.0,54.0,12.0,NaN,NaN,NaN,NaN
cd696cdeb3ac,NO,YES,1.0,10.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,NO,YES,1.0,49.0,1.0,NaN,NaN,NaN,NaN
e102508752e8,NO,YES,2.0,45.0,2.0,NaN,NaN,NaN,NaN
b93d164cbee8,NO,YES,2.0,9.0,3.0,NaN,NaN,NaN,NaN


In [16]:
def recode_sex(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "MALE"
        elif value == 2:
            return "FEMALE"
        else:
            return np.nan

In [17]:
clean_columns(working_df, ['sex'], [recode_sex])
working_df

/Users/kathrynpanger/Documents/swb_inspect_data/cleaning_functions/clean_csv.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: function(x))


,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,NO,YES,MALE,63.0,1.0,NaN,NaN,NaN,NaN
58b6880790bd,NO,YES,FEMALE,59.0,2.0,NaN,NaN,NaN,NaN
d2a2b4eb1d80,NO,YES,FEMALE,13.0,3.0,NaN,NaN,NaN,NaN
9dadc3d1eff3,NO,YES,FEMALE,54.0,12.0,NaN,NaN,NaN,NaN
cd696cdeb3ac,NO,YES,MALE,10.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,NO,YES,MALE,49.0,1.0,NaN,NaN,NaN,NaN
e102508752e8,NO,YES,FEMALE,45.0,2.0,NaN,NaN,NaN,NaN
b93d164cbee8,NO,YES,FEMALE,9.0,3.0,NaN,NaN,NaN,NaN


In [18]:
def recode_relationship_to_head(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "HEAD"
        elif value == 2:
            return "SPOUSE"
        elif value == 3:
            return "OWN CHILD"
        elif value == 4:
            return "STEP CHILD"
        elif value == 5:
            return "ADOPTED CHILD"
        elif value == 6:
            return "GRANDCHILD"
        elif value == 7:
            return "SIBLING"
        elif value == 8:
            return 'NIECE OR NEPHEW'
        elif value == 9:
            return "SIBLING-IN-LAW"
        elif value == 10:
            return "PARENT"
        elif value == 11:
            return "PARENT-IN-LAW"
        elif value == 12:
            return "DOMESTIC HELP"
        elif value == 14:
            return "OTHER RELATION"
        elif value == 15:
            return "OTHER NON-RELATION"
        elif value == 16:
            return "CHILD-IN-LAW"
        elif value == 98:
            return 'FORMER_HEAD'
        else:
            return np.nan

In [19]:

clean_columns(working_df, ['relationship_to_head'], [recode_relationship_to_head])
working_df

/Users/kathrynpanger/Documents/swb_inspect_data/cleaning_functions/clean_csv.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: function(x))


,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,NO,YES,MALE,63.0,HEAD,NaN,NaN,NaN,NaN
58b6880790bd,NO,YES,FEMALE,59.0,SPOUSE,NaN,NaN,NaN,NaN
d2a2b4eb1d80,NO,YES,FEMALE,13.0,OWN CHILD,NaN,NaN,NaN,NaN
9dadc3d1eff3,NO,YES,FEMALE,54.0,DOMESTIC HELP,NaN,NaN,NaN,NaN
cd696cdeb3ac,NO,YES,MALE,10.0,OWN CHILD,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,NO,YES,MALE,49.0,HEAD,NaN,NaN,NaN,NaN
e102508752e8,NO,YES,FEMALE,45.0,SPOUSE,NaN,NaN,NaN,NaN
b93d164cbee8,NO,YES,FEMALE,9.0,OWN CHILD,NaN,NaN,NaN,NaN


In [20]:
def recode_why_join_household(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "NEW BORN"
        elif value == 2:
            return "ADOPTED CHILD"
        elif value == 3:
            return "MARRIAGE OR COHABITATION"
        elif value == 4:
            return "DIVORCE OR SEPARATION"
        elif value == 5:
            return "RETURNED FROM COLLEGE"
        elif value == 6:
            return "RETURNED FROM INSTITUTION"
        elif value == 7:
            return "MOVED IN WITH RELATIVE"
        elif value == 8:
            return 'SHARED ACCOMMODATION'
        elif value == 9:
            return "RETURN FROM WORK MIGRATION"
        elif value == 10:
            return "MISTAKENLY NOT REPORTED OR FORGOTTEN LAST VISIT"
        elif value == 11:
            return "DISPLACED DUE TO CONFLICT"
        elif value == 12:
            return "CORONAVIRUS RELATED"
        elif value == 15:
            return "DISPLACED DUE TO DROUGHT"
        elif value == 16:
            return "HARVEST LOSS"
        elif value == 17:
            return "DISPLACED DUE TO FLOOD"
        elif value == 96:
            return 'OTHER'
        else:
            return np.nan

In [21]:
clean_columns(working_df, ['why_join_household'], [recode_why_join_household])
working_df

/Users/kathrynpanger/Documents/swb_inspect_data/cleaning_functions/clean_csv.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(lambda x: function(x))


,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,NO,YES,MALE,63.0,HEAD,NaN,NaN,NaN,NaN
58b6880790bd,NO,YES,FEMALE,59.0,SPOUSE,NaN,NaN,NaN,NaN
d2a2b4eb1d80,NO,YES,FEMALE,13.0,OWN CHILD,NaN,NaN,NaN,NaN
9dadc3d1eff3,NO,YES,FEMALE,54.0,DOMESTIC HELP,NaN,NaN,NaN,NaN
cd696cdeb3ac,NO,YES,MALE,10.0,OWN CHILD,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,NO,YES,MALE,49.0,HEAD,NaN,NaN,NaN,NaN
e102508752e8,NO,YES,FEMALE,45.0,SPOUSE,NaN,NaN,NaN,NaN
b93d164cbee8,NO,YES,FEMALE,9.0,OWN CHILD,NaN,NaN,NaN,NaN


In [22]:
working_df['age'] = working_df['age'].astype("Int64")
working_df

/var/folders/lw/jy_c2z0x221fsb78kk1_h2800000gn/T/ipykernel_14590/391576009.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['age'] = working_df['age'].astype("Int64")


,is_new_household_member,is_still_household_member,sex,age,relationship_to_head,other_relationship,why_join_household,other_reason_for_joining,is_selected_farmer
individual_id,,,,,,,,,
db95a04273b3,NO,YES,MALE,63,HEAD,NaN,NaN,NaN,NaN
58b6880790bd,NO,YES,FEMALE,59,SPOUSE,NaN,NaN,NaN,NaN
d2a2b4eb1d80,NO,YES,FEMALE,13,OWN CHILD,NaN,NaN,NaN,NaN
9dadc3d1eff3,NO,YES,FEMALE,54,DOMESTIC HELP,NaN,NaN,NaN,NaN
cd696cdeb3ac,NO,YES,MALE,10,OWN CHILD,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
7b559a072842,NO,YES,MALE,49,HEAD,NaN,NaN,NaN,NaN
e102508752e8,NO,YES,FEMALE,45,SPOUSE,NaN,NaN,NaN,NaN
b93d164cbee8,NO,YES,FEMALE,9,OWN CHILD,NaN,NaN,NaN,NaN


In [23]:
save_string = f"../data/processed/individual_id_questions/{dfs[0][1]}"
# working_df.to_csv(save_string)

In [37]:
working_df = dfs[1][0]
working_df_name = dfs[1][1]
print(working_df_name)
working_df

p2r3_sect_2_6b_6c.csv


,zone,state,lga,sector,ea,hhid,indiv,s2q2,s2q3,s2q4,...,s6cq1_1,s6cqe2_1,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,s6cq5_1,s6cqe3_1,s6cq6_1,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,2,1.0,NaN,...,18.0,1.0,1.0,NaN,NaN,3.0,44.0,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,2,1.0,NaN,...,99.0,1.0,2.0,5.0,9.0,3.0,NaN,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,2,1.0,NaN,...,99.0,2.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,2,1.0,NaN,...,16.0,1.0,2.0,4.0,4.0,6.0,NaN,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,2,1.0,NaN,...,99.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
rename_dict= {'s2q2': 'is_new_household_member',
              's2q3': 'is_still_household_member',
              's2q5': 'sex',
              's2q6': 'age',
              's2q7': 'relationship_to_head',
              's2q7_os': 'other_relationship',
              's2q8': 'why_join_household',
              's2q8_os':'other_reason_for_joining',
              's2q14': 'is_covid_vaccinated',
              's6q0b_1': 'is_available_to_respond',
              's6q1_1': 'worked_for_pay_last_week',
              's6q1a_1': 'has_business_was_absent_last_week',
              's6q1b_1': 'when_expected_to_return_to_work',
              's6q1c_1': 'why_not_work_last_week',
              's6q1c_os_1': 'other_reason_not_work',
              's6q3a_1': 'tried_to_earn_pay_last_month',
              's6q3b_1': 'how_tried_to_earn_pay_last_month',
              's6q3b_os_1': 'other_way_tried_to_earn_pay_last_month',
              's6q5b_1': 'main_activity_of_employers_business_last_week',
              's6q6a_1': "family_farm_food_fate",
              "s6q8b1_1": 'hours_worked_last_week_main_business_activity',
              's6q9a_1': 'worked_december_2021',
              's6q9b_1': 'worked_january_2022',
              's6q9c_1': 'worked_february_2022',
              's6q9d_1': 'worked_march_2022',
              's6cq1_1': 'age_started_working',
              's6cqe2_1': 'is_employed',
              's6cq5_1': 'years_worked_first_job',
              's6cq6_1': 'why_left_first_job'
              }
working_df = working_df.rename(columns = rename_dict)
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,2,1.0,NaN,...,18.0,1.0,1.0,NaN,NaN,3.0,44.0,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,2,1.0,NaN,...,99.0,1.0,2.0,5.0,9.0,3.0,NaN,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,2,1.0,NaN,...,99.0,2.0,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,2,1.0,NaN,...,16.0,1.0,2.0,4.0,4.0,6.0,NaN,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,2,1.0,NaN,...,99.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,2,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Recode  the binary columns and all shared variables from the previous dataset

#binary variables
binary_columns = ['is_new_household_member',
                  'is_still_household_member',
                  'is_covid_vaccinated',
                  'is_available_to_respond',
                  'worked_for_pay_last_week',
                  'has_business_was_absent_last_week',
                  'tried_to_earn_pay_last_month',
                  'worked_december_2021',
                  'worked_january_2022',
                  'worked_february_2022',
                  'worked_march_2022',
                  'is_employed'
                  ]
# age and sex
clean_columns(working_df, selected_columns=binary_columns, cleaning_functions_list=[recode_binary])
working_df['age'] = working_df['age'].astype("Int64")
working_df['age_started_working'] = working_df['age'].astype("Int64")
working_df['years_worked_first_job'] = working_df['age'].astype("Int64")
clean_columns(working_df, ['sex'], [recode_sex])

# Relationship to head
clean_columns(working_df, ['relationship_to_head'], [recode_relationship_to_head])

#Why Join household
clean_columns(working_df, ['why_join_household'], [recode_why_join_household])

working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [40]:
# recode when_expected_to_return_to_work

def recode_when_expected_to_return_to_work(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "WITHIN ONE WEEK"
        elif value == 2:
            return "WITHIN ONE MONTH"
        elif value == 3:
            return "WITHIN THREE MONTHS"
        elif value == 4:
            return "MORE THAN THREE MONTHS"
        elif value == 5:
            return "ONCE RESTRICTIONS ARE LIFTED"
        elif value == 98:
            return "DO NOT KNOW"
        else:
            return np.nan
        
clean_columns(working_df, ['when_expected_to_return_to_work'], [recode_when_expected_to_return_to_work])

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [41]:
def recode_why_not_work_last_week(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "CORONAVIRUS CLOSURE"
        elif value == 2:
            return "CLOSURE FOR NON-CORONAVIRUS REASON"
        elif value == 3:
            return "LAID OFF"
        elif value == 4:
            return "FURLOUGHED"
        elif value == 5:
            return "VACATION"
        elif value == 6:
            return "SICK"
        elif value == 7:
            return "CARE FOR ILL RELATIVE"
        elif value == 8:
            return 'SEASONAL WORKER'
        elif value == 9:
            return "RETIRED"
        elif value == 10:
            return "MOVEMENT RESTRICTIONS"
        elif value == 11:
            return "LACK OF INPUTS FOR FARMING"
        elif value == 12:
            return "NOT FARMING SEASON"
        elif value == 13:
            return "PERSONNEL ROTATION DUE TO CORONAVIRUS"
        elif value == 14:
            return "CONFLICT OR INSURGENCY"
        elif value == 15:
            return "CLOSED DUE TO ENDSARS PROTESTS"
        elif value == 17:
            return 'LACK OF BUSINESS INPUTS'
        elif value == 18:
            return "MATERNITY LEAVE"
        elif value == 96:
            return 'OTHER'
        else:
            return np.nan
        
clean_columns(working_df, ['why_not_work_last_week'], [recode_why_not_work_last_week])

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [42]:
def recode_how_tried_to_earn_pay_last_month(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "APPLY TO PROSPECTIVE EMPLOYERS"
        elif value == 2:
            return "ANSWER JOB ADVERTISEMENTS"
        elif value == 3:
            return "STUDY JOB ADVERTISEMENTS"
        elif value == 4:
            return "REGISTER WITH UNEMPLOYMENT CENTER"
        elif value == 5:
            return "REGISTER WITH PRIVATE RECRUITMENT OFFICE"
        elif value == 6:
            return "TAKE A TEST OR INTERVIEDW"
        elif value == 7:
            return "SEEK HELP FROM RELATIVES, FRIENDS, OTHERS"
        elif value == 8:
            return 'CHECK AT WORK SITES'
        elif value == 9:
            return "WAIT ON TEH STREET TO BE RECRUITED"
        elif value == 10:
            return "SEEK FINANCIAL HELP TO START A BUSINESS"
        elif value == 11:
            return "LOOK FOR NON-FINANCIAL RESOURCES TO START A BUSINESS"
        elif value == 12:
            return "APPLY FOR PERMIT TO START A BUSINESS"
        elif value == 96:
            return "OTHER"
        else:
            return np.nan
clean_columns(working_df, ['how_tried_to_earn_pay_last_month'], [recode_how_tried_to_earn_pay_last_month])

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [43]:
def recode_main_activity_of_employers_business_last_week(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "AGRICULTURE, HUNTING, FISHING"
        elif value == 2:
            return "MINING, MANUFACTURING"
        elif value == 3:
            return "ELECTRICITY, GAS, WATER SUPPLY"
        elif value == 4:
            return "CONSTRUCTION"
        elif value == 5:
            return "BUYING & SELLING GOODS, REPAIR OF GOODS, HOTELS & RESTAURANTS"
        elif value == 6:
            return "TRANSPORT, DRIVING, POST, TRAVEL AGENCIES"
        elif value == 7:
            return "PROFESSIONAL ACTIVITIES: FINANCE, LEGAL, ANALYSIS, COMPUTER, REAL ESTATE"
        elif value == 8:
            return 'PUBLIC ADMINISTRATION'
        elif value == 9:
            return "PERSONAL SERVICES, EDUCATION, HEALTH, CULTURE, SPORT, DOMESTIC WORK, OTHER"
        else:
            return np.nan
clean_columns(working_df, ['main_activity_of_employers_business_last_week'], [recode_main_activity_of_employers_business_last_week])
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [44]:
def recode_family_farm_food_fate(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "ONLY FOR SALE"
        elif value == 2:
            return "ONLY FOR FAMILY CONSUMPTION"
        elif value == 3:
            return "BOTH FAMILY AND SALE"
        else:
            return np.nan
clean_columns(working_df, ['family_farm_food_fate'], [recode_family_farm_food_fate])
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [45]:
def recode_hours_worked_last_week_main_business_activity(entry):
    if math.isnan(entry) or int(entry) != 998:
        return entry
    #If not, recode the variable this way
    else:
        return "DO NOT KNOW"
    
clean_columns(working_df, ['hours_worked_last_week_main_business_activity'], [recode_hours_worked_last_week_main_business_activity])
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,6.0,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,2.0,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [46]:
def recode_why_left_first_job(entry):
    if math.isnan(entry):
        return entry
    #If not, recode the variable this way
    else:
        value = int(entry)
        if value == 1:
            return "TEMPORARY CONTRACT"
        elif value == 2:
            return "TO HAVE A BETTER PAID JOB"
        elif value == 3:
            return "TO HAVE MORE CLIENTS OR BUSINESS"
        elif value == 4:
            return "TO WORK MORE HOURS"
        elif value == 5:
            return "TO WORK FEWER HOURS"
        elif value == 6:
            return "TO BETTER MATCH SKILLS"
        elif value == 7:
            return "TO WORK CLOSER TO HOME"
        elif value == 8:
            return "TO IMPROVE OTHER WORKING CONDITIONS"  
        elif value == 9:
            return "LAID OFF"
        elif value == 10:
            return "BUSINESS CLOSED"
        elif value == 11:
            return "TO RELOCATE"        
        elif value == 12:
            return "TO STUDY OR GAIN NEW SKILLS"      
        elif value == 96:
            return "OTHER"
        else:
            return np.nan
clean_columns(working_df, ['why_left_first_job'], [recode_why_left_first_job])
working_df

,zone,state,lga,sector,ea,hhid,indiv,is_new_household_member,is_still_household_member,s2q4,...,age_started_working,is_employed,s6cq1b_1,s6cq2_1,s6cq3_1,s6cq4_1,years_worked_first_job,s6cqe3_1,why_left_first_job,s6cq6_os_1
individual_id,,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,NO,YES,NaN,...,63,YES,1.0,NaN,NaN,3.0,63,1.0,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,NO,YES,NaN,...,42,YES,2.0,5.0,9.0,3.0,42,1.0,TO BETTER MATCH SKILLS,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,NO,YES,NaN,...,12,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,NO,YES,NaN,...,54,NO,NaN,1.0,NaN,NaN,54,2.0,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,NO,YES,NaN,...,49,YES,2.0,4.0,4.0,6.0,49,1.0,TO HAVE A BETTER PAID JOB,NaN
e102508752e8,1,37,3702,1,1102,379155,2,NO,YES,NaN,...,45,YES,NaN,NaN,NaN,NaN,45,1.0,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,NO,YES,NaN,...,9,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN


In [48]:
working_df = dfs[2][0]
working_df_name = dfs[2][1]
working_df_name

'p2r8_sect_2.csv'

In [49]:
working_df 

,zone,state,lga,sector,ea,hhid,indiv,s2q2,s2q3,s2q4,s2q4_os,s2q5,s2q6,s2q7a,s2q7,s2q7_os,s2q8,s2q8_os,s2q9,s2q9_os
individual_id,,,,,,,,,,,,,,,,,,,,
db95a04273b3,4,1,115,1,670,10001,1,2,1.0,NaN,NaN,1.0,63.0,1,1.0,NaN,NaN,NaN,NaN,NaN
58b6880790bd,4,1,115,1,670,10001,2,2,1.0,NaN,NaN,2.0,59.0,1,2.0,NaN,NaN,NaN,NaN,NaN
d2a2b4eb1d80,4,1,115,1,670,10001,3,2,1.0,NaN,NaN,2.0,13.0,1,3.0,NaN,NaN,NaN,NaN,NaN
9dadc3d1eff3,4,1,115,1,670,10001,4,2,1.0,NaN,NaN,2.0,54.0,1,12.0,NaN,NaN,NaN,NaN,NaN
cd696cdeb3ac,4,1,115,1,670,10001,8,2,1.0,NaN,NaN,1.0,10.0,1,3.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7b559a072842,1,37,3702,1,1102,379155,1,2,1.0,NaN,NaN,1.0,49.0,1,1.0,NaN,NaN,NaN,NaN,NaN
e102508752e8,1,37,3702,1,1102,379155,2,2,1.0,NaN,NaN,2.0,45.0,1,2.0,NaN,NaN,NaN,NaN,NaN
b93d164cbee8,1,37,3702,1,1102,379155,3,2,1.0,NaN,NaN,2.0,9.0,1,3.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
working_df = dfs[3][0]
working_df_name = dfs[3][1]
working_df_name